In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load preprocessed data
train_images = np.load('../../data/processed/train_images.npy')
train_labels = np.load('../../data/processed/train_labels.npy')
test_images = np.load('../../data/processed/test_images.npy')
test_labels = np.load('../../data/processed/test_labels.npy')

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

In [3]:
# Define CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # Number of mood classes
])

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [6]:
# Train the model
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    validation_data=(test_images, test_labels),
                    epochs=20,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

Epoch 1/20
898/898 [==============================] - 76s 84ms/step - loss: 1.7990 - accuracy: 0.2516 - val_loss: 1.7273 - val_accuracy: 0.2874
Epoch 2/20
898/898 [==============================] - 83s 92ms/step - loss: 1.7520 - accuracy: 0.2764 - val_loss: 1.6986 - val_accuracy: 0.3040
Epoch 3/20
898/898 [==============================] - 89s 99ms/step - loss: 1.7091 - accuracy: 0.3022 - val_loss: 1.5585 - val_accuracy: 0.3828
Epoch 4/20
898/898 [==============================] - 79s 88ms/step - loss: 1.6306 - accuracy: 0.3504 - val_loss: 1.4286 - val_accuracy: 0.4585
Epoch 5/20
898/898 [==============================] - 80s 89ms/step - loss: 1.5556 - accuracy: 0.3917 - val_loss: 1.4169 - val_accuracy: 0.4465
Epoch 6/20
898/898 [==============================] - 82s 91ms/step - loss: 1.5087 - accuracy: 0.4121 - val_loss: 1.3405 - val_accuracy: 0.4889
Epoch 7/20
898/898 [==============================] - 80s 89ms/step - loss: 1.4709 - accuracy: 0.4319 - val_loss: 1.3097 - val_accuracy:

In [7]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Test Accuracy: {test_acc}')

225/225 - 3s - loss: 1.2091 - accuracy: 0.5419 - 3s/epoch - 12ms/step
Test Accuracy: 0.5419337153434753


In [8]:
# Save the trained model
model.save('../../models/mood_cnn_model.h5')